In [13]:
import sys, os, importlib
sys.path.append(os.path.abspath(".."))

from src import fetch_data
importlib.reload(fetch_data)

from src.fetch_data import list_tables, fetch_table
import delta_sharing
import time
from datetime import datetime

def robust_fetch_table(share, schema, table, max_retries=3, delay=2):
    """
    Robust table fetching with retry logic for authentication issues
    """
    for attempt in range(max_retries):
        try:
            print(f"  Attempt {attempt + 1}/{max_retries}...")
            
            # Create fresh client for each attempt
            client = delta_sharing.SharingClient('../credentials/config.share')
            
            # Get fresh table listing
            tables = client.list_all_tables()
            target_table = None
            for t in tables:
                if t.share == share and t.schema == schema and t.name == table:
                    target_table = t
                    break
            
            if not target_table:
                raise Exception(f"Table {share}.{schema}.{table} not found in fresh listing")
            
            # Try to fetch with immediate execution
            df = fetch_table(share, schema, table, save=True)
            return df
            
        except Exception as e:
            error_msg = str(e)
            if "FileNotFoundError" in str(type(e)) or "401" in error_msg or "NoAuthenticationInformation" in error_msg:
                if attempt < max_retries - 1:
                    print(f"    Authentication/URL issue, retrying in {delay}s...")
                    time.sleep(delay)
                    continue
                else:
                    print(f"    Final attempt failed: {type(e).__name__}")
                    raise e
            else:
                print(f"    Non-retryable error: {type(e).__name__}: {error_msg[:100]}...")
                raise e
    
    return None

# Step 1: List available tables
print(" Listing available tables...")
tables = list_tables()
print(f"Found {len(tables)} tables:")
for t in tables:
    print(f"  - {t.share}.{t.schema}.{t.name}")

print(f"\nCurrent time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


print("\n Attempting to fetch tables with retry logic...")
all_dfs = {}
failed_tables = []

for t in tables:
    key = f"{t.share}.{t.schema}.{t.name}"
    print(f"\n Fetching {key}...")
    
    try:
        df = robust_fetch_table(t.share, t.schema, t.name)
        all_dfs[key] = df
        print(f" Successfully fetched {key} - Shape: {df.shape}")
        
    except Exception as e:
        print(f" Failed to fetch {key} after all retries")
        failed_tables.append((key, str(e)))

# Step 3: Report results
print(f"\n Summary:")
print(f"   Successfully fetched: {len(all_dfs)} tables")
print(f"   Failed: {len(failed_tables)} tables")

if failed_tables:
    print(f"\n Failed tables:")
    for table, error in failed_tables:
        print(f"  - {table}: {error[:100]}...")

# Step 4: Preview successful data
if all_dfs:
    print(f"\n Data Preview:")
    for table_name, df in all_dfs.items():
        print(f"\n{table_name} (Shape: {df.shape}):")
        print(df.head(3))
        print(f"Columns: {list(df.columns)}")
else:
    print(f"\n  No data was successfully fetched.")
    print(f"This could be due to:")
    print(f"  - Authentication issues with the Delta Sharing provider")
    print(f"  - Network connectivity problems")
    print(f"  - Expired or invalid credentials")
    print(f"  - Provider-side configuration issues")


 Listing available tables...
Found 3 tables:
  - payments_prod.tables.payments_item_summary
  - commerce_prod.tables.commerce_item_details
  - commerce_prod.tables.commerce_tax_details

Current time: 2025-09-10 11:31:50

 Attempting to fetch tables with retry logic...

 Fetching payments_prod.tables.payments_item_summary...
  Attempt 1/3...
    Authentication/URL issue, retrying in 2s...
  Attempt 2/3...
    Authentication/URL issue, retrying in 2s...
  Attempt 3/3...
    Final attempt failed: FileNotFoundError
 Failed to fetch payments_prod.tables.payments_item_summary after all retries

 Fetching commerce_prod.tables.commerce_item_details...
  Attempt 1/3...
    Authentication/URL issue, retrying in 2s...
  Attempt 2/3...
    Authentication/URL issue, retrying in 2s...
  Attempt 3/3...
    Final attempt failed: FileNotFoundError
 Failed to fetch commerce_prod.tables.commerce_item_details after all retries

 Fetching commerce_prod.tables.commerce_tax_details...
  Attempt 1/3...
    Au

In [ ]:
# 🔧 Diagnostic Information
# Run this cell if you're still having issues to get detailed diagnostic info

import json
import requests
from pathlib import Path
from datetime import datetime

print("🔧 Delta Sharing Diagnostic Information")
print("=" * 50)

# 1. Check credential file
print("\n1. Credential File Status:")
# Fix path - notebook runs from notebooks/ directory, so go up one level
cred_file = Path("../credentials/config.share")
if cred_file.exists():
    with open(cred_file) as f:
        creds = json.load(f)
    print(f"    File exists")
    print(f"    Expiration: {creds.get('expirationTime', 'Not specified')}")
    print(f"    Endpoint: {creds.get('endpoint', 'Not specified')[:50]}...")
else:
    print(f"    File not found at {cred_file}")
    # Also check absolute path
    abs_cred_file = Path("/Users/vishwaspatel/data_viz_app/credentials/config.share")
    if abs_cred_file.exists():
        print(f"    Found at absolute path: {abs_cred_file}")
        with open(abs_cred_file) as f:
            creds = json.load(f)
        print(f"    Expiration: {creds.get('expirationTime', 'Not specified')}")
        print(f"    Endpoint: {creds.get('endpoint', 'Not specified')[:50]}...")

# 2. Check network connectivity
print("\n2. Network Connectivity:")
try:
    response = requests.get("https://eastus-c3.azuredatabricks.net", timeout=10)
    print(f"    Databricks endpoint reachable (Status: {response.status_code})")
except Exception as e:
    print(f"    Databricks endpoint unreachable: {e}")

# 3. Check Delta Sharing version
print("\n3. Delta Sharing Setup:")
try:
    import delta_sharing
    print(f"    Delta Sharing version: {delta_sharing.__version__}")
except Exception as e:
    print(f"    Delta Sharing import failed: {e}")

# 4. Test basic client creation
print("\n4. Client Creation Test:")
try:
    # Use the correct path for client creation
    client_path = "../credentials/config.share" if Path("../credentials/config.share").exists() else "/Users/vishwaspatel/data_viz_app/credentials/config.share"
    client = delta_sharing.SharingClient(client_path)
    print(f"    Client created successfully")
    
    # Test table listing
    tables = client.list_all_tables()
    print(f"    Table listing successful ({len(tables)} tables found)")
    
except Exception as e:
    print(f"    Client creation failed: {e}")

# 5. Current time vs URL expiration
print(f"\n5. Time Information:")
print(f"    Current time: {datetime.now().isoformat()}")
print(f"     URLs typically expire within 1-2 hours of generation")

print("\n" + "=" * 50)
print("💡 If all checks pass but data fetching fails, the issue is likely:")
print("   - Provider-side authentication configuration")
print("   - Network firewall/proxy blocking Azure Data Lake access")
print("   - Temporary service issues on the provider side")


🔧 Delta Sharing Diagnostic Information

1. Credential File Status:
   ✅ File exists
   📅 Expiration: 2027-08-31T17:53:24.389Z
   🌐 Endpoint: https://eastus-c3.azuredatabricks.net/api/2.0/delt...

2. Network Connectivity:
   ✅ Databricks endpoint reachable (Status: 200)

3. Delta Sharing Setup:
   ✅ Delta Sharing version: 1.3.3

4. Client Creation Test:
   ✅ Client created successfully
   ✅ Table listing successful (3 tables found)

5. Time Information:
   🕐 Current time: 2025-09-10T10:59:00.358601
   ⚠️  URLs typically expire within 1-2 hours of generation

💡 If all checks pass but data fetching fails, the issue is likely:
   - Provider-side authentication configuration
   - Network firewall/proxy blocking Azure Data Lake access
   - Temporary service issues on the provider side
